In [32]:
import pandas as pd
import numpy as np
import os
from keras.models import load_model
from keras.preprocessing.text import Tokenizer

In [33]:
df = pd.read_csv('./dataset/lcquad2/lcquad2.csv').iloc[:, 0:11] 
df.head()

,NNQT_question,answer,paraphrased_question,question,sparql_dbpedia18,sparql_wikidata,subgraph,template,template_id,template_index,uid
0,Did {Alexander_Hamilton} {occupation} {lawyer}?,[],Did Alexander Hamilton practice law?,Is Alexander Hamilton a lawyer?,ASK { ?statement1 <http://www.w3.org/1999/02/...,ASK WHERE { wd:Q178903 wdt:P106 wd:Q40348 },boolean one_hop right subgraph,Ask (ent-pred-obj),1,0,0
1,Did {FC_Barcelona} {chairperson} {Juan José Ib...,[],Does FC Barcelona have Juan Jose Ibarretxe as ...,Is Juan José Ibarretxe a chairperson of FC Bar...,ASK { ?statement1 <http://www.w3.org/1999/02/...,ASK WHERE { wd:Q7156 wdt:P488 wd:Q351738 },boolean one_hop right subgraph,Ask (ent-pred-obj`),1,1,1
2,Did {Woody_Harrelson} {family name} {Harrelson}?,[],Is Harrelson the real family name of Woody Har...,Is Woody Harrelson's last name really Harrelson?,ASK { ?statement1 <http://www.w3.org/1999/02/...,ASK WHERE { wd:Q201279 wdt:P734 wd:Q21450338 },boolean one_hop right subgraph,Ask (ent-pred-obj),1,2,2
3,Did {Ivanka_Trump} {occupation} {television pr...,[],Was Ivonka Trump ever a tv host?,Did Ivanka Trump host a tv show?,ASK { ?statement1 <http://www.w3.org/1999/02/...,ASK WHERE { wd:Q239411 wdt:P106 wd:Q947873 },boolean one_hop right subgraph,Ask (ent-pred-obj),1,3,3
4,Did {Rihanna} {record label} {Motown}?,[],Has Rihanna ever recorded for Motown?,Did Rihanna recorded the label of Motown?,ASK { ?statement1 <http://www.w3.org/1999/02/...,ASK WHERE { wd:Q36844 wdt:P264 wd:Q43327 },boolean one_hop right subgraph,Ask (ent-pred-obj`),1,4,4


In [34]:
df['question'].isnull().sum()

766

In [35]:
df.dropna(subset=['question'],inplace=True)

In [36]:
X_predict = df.question

In [ ]:
df2 = pd.read_csv('./dataset/PosNeg.csv')
title = df2.Question

In [37]:
# take tokens and build word-in dictionary
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")
tokenizer.fit_on_texts(title)

In [50]:
# X_predict = ["who was the american general in the pacific during world war ii","where do guyanese people live","what is magic johnsons dads name"]
model = load_model('./model/DenseModel.h5')
x_predict_word_ids = tokenizer.texts_to_sequences(X_predict)
x_predict = tokenizer.sequences_to_matrix(x_predict_word_ids, mode='binary')
predict_test = model.predict(x_predict)
predict_result = np.argmax(predict_test,axis=1)                           # 1 temporal   0 no-temporal
print(predict_result)
print(len(predict_result))

[0 0 0 ... 0 1 0]
30279


In [51]:
f0 = open("./dataset/lcquad2/output/no-temporal.txt", "w")
f1 = open("./dataset/lcquad2/output/temporal.txt", "w")
for i in range(0, len(X_predict)):
    if predict_result[i] == 0:
        f0.write(X_predict.iloc[i] + "\n")
    elif predict_result[i] == 1:
        f1.write(X_predict.iloc[i] + "\n")
f0.close()
f1.close()